In [ ]:
# !pip install \
#     --extra-index-url=https://pypi.nvidia.com \
#     dask-cudf-cu12==24.4.* cuml-cu12==24.4.*
    
# !pip install imblearn
# !pip install dask-ml
# !pip install tabulate nb_black
# !pip install -U ray
# !pip install ray[tune]
# !pip install bayesian-optimization scikit-optimize

In [ ]:
import os 
import ray
import multiprocessing



from DataLoader import getPreprocessedData
from DataLoader import getPreprocessedDataOnGPU
from HPO import build_Bayes_search_alg,select_sched_alg


from ray import tune

: 

# Getting Data ready

In [2]:
trainX, trainY, testX, testY = getPreprocessedDataOnGPU()

RuntimeError: Fatal CUDA error encountered at: /__w/cudf/cudf/cpp/src/io/utilities/datasource.cpp:123: 35 cudaErrorInsufficientDriver CUDA driver version is insufficient for CUDA runtime version

In [2]:
import cuml

ImportError: libcublas.so.12: cannot open shared object file: No such file or directory

# Settings For Experiment 

In [17]:
num_samples = 50
compute = "GPU"
CV_folds = 3 # The number of Cross-Validation folds to be performed
search_alg = "BayesOpt"  # Options: SkOpt or BayesOpt
sched_alg = "AsyncHyperBand"  # Options: AsyncHyperBand or MedianStop
max_concurrent_gpus = 1 # number of gpus to play in experiemnt  -1 to use them all 
cpu_per_sample = int(multiprocessing.cpu_count() / max_concurrent_gpus)

# HPO params ranges 
n_estimators_range = (1, 100)
max_depth_range = (2, 15)
max_features_range = (0.1, 0.8)

hpo_ranges = {
    "n_estimators": n_estimators_range,
    "max_depth": max_depth_range,
    "max_features": max_features_range,
}

In [18]:
# connecting to localhost of dash bording and experiment tracking 
#ray.init(local_mode=True,num_gpus=max_concurrent_gpus)

In [19]:
# init search algo 
search = build_Bayes_search_alg(search_alg, hpo_ranges)
# init sechduler algo 
sched = select_sched_alg(sched_alg)

In [23]:
exp_name = None 
if exp_name is not None:
    exp_name += exp_name
else:
    exp_name = ""
    exp_name += "{}_{}_CV-{}M_SAMP-{}".format(
        "RF", compute , CV_folds, num_samples
    )
    
    exp_name += "_{}".format("Random" if search_alg is None else search_alg)
    
    if sched_alg is not None : 
        exp_name += "_{}".format(sched_alg)
        
        
static_config = {
    "compute": compute,
    "num_workers": cpu_per_sample if compute == "CPU" else 1,
}

In [25]:
results_dir = "results"

In [ ]:
analysis = tune.run(
    WrappedTrainable,
    name=exp_name,
    scheduler=sched,
    search_alg=search,
    stop={
        "training_iteration": CV_folds,
        "is_bad": True,
    },
    resources_per_trial={"cpu": cpu_per_sample, "gpu": int(compute == "GPU")},
    num_samples=num_samples,
    checkpoint_at_end=True,
    keep_checkpoints_num=1,
    local_dir=results_dir,
    trial_name_creator=get_trial_name,
    checkpoint_score_attr="test_accuracy",
    verbose=1,
    raise_on_failed_trial=False,
)